# HDS5210-2022 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 23rd.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [56]:
def priest(sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_bp, temperature, alertness, inspired_oxygen, performance_status):
    """
    Calculate the PRIEST Clinical Severity Score based on the provided parameters.

    Parameters:
    - sex (str): Gender assigned at birth, can be 'male' or 'female' (not case-sensitive).
    - age (int): Age in years.
    - respiratory_rate (int): Respiratory rate in breaths per minute.
    - oxygen_saturation (float): Oxygen saturation as a percent between 0 and 1.
    - heart_rate (int): Heart rate in beats per minute.
    - systolic_bp (int): Systolic BP in mmHg.
    - temperature (float): Temperature in degrees C.
    - alertness (str): Alertness level, can be 'Alert' or 'Confused or not alert' (not case-sensitive).
    - inspired_oxygen (str): Type of oxygen, can be 'Air' or 'Supplemental oxygen' (not case-sensitive).
    - performance_status (str): Performance status description.

    Returns:
    - float: Risk percentage between 0 and 1. Returns None for invalid inputs.

    Example:
    >>> priest("male", 45, 20, 0.95, 80, 110, 37.0, "Alert", "Air", "Unrestricted normal activity")
    0.02
    >>> priest("female", 85, 25, 0.91, 100, 85, 39.5, "Confused or not alert", "Supplemental oxygen", "Bed/chair bound, no self-care")
    0.59
    >>> priest("male", 50, 12, 0.98, 60, 120, 36.5, "Alert", "Air", "Limited strenuous activity, can do light activity")
    0.03
    """

    # Scoring for each parameter
    score = 0

    # Score based on sex
    score += 1 if sex.lower() == "male" else 0

    # Score based on age
    if 50 <= age <= 65:
        score += 2
    elif 66 <= age <= 80:
        score += 3
    elif age > 80:
        score += 4

    # Score based on respiratory rate
    if respiratory_rate < 9 or respiratory_rate > 24:
        score += 3
    elif 9 <= respiratory_rate <= 11 or 21 <= respiratory_rate <= 24:
        score += 1
    elif 12 <= respiratory_rate <= 20:
        score += 0

    # Score based on oxygen saturation
    if oxygen_saturation > 0.95:
        score += 0
    elif 0.94 <= oxygen_saturation <= 0.95:
        score += 1
    elif 0.92 <= oxygen_saturation <= 0.93:
        score += 2
    elif oxygen_saturation < 0.92:
        score += 3

    # Score based on heart rate
    if heart_rate < 41 or heart_rate > 130:
        score += 3
    elif 41 <= heart_rate <= 50 or 111 <= heart_rate <= 130:
        score += 1
    elif 51 <= heart_rate <= 90:
        score += 0
    elif 91 <= heart_rate <= 110:
        score += 1

    # Score based on systolic blood pressure
    if systolic_bp < 91 or systolic_bp > 219:
        score += 3
    elif 91 <= systolic_bp <= 100:
        score += 2
    elif 101 <= systolic_bp <= 110:
        score += 1
    elif 111 <= systolic_bp <= 219:
        score += 0

    # Score based on temperature
    if temperature < 35.1:
        score += 3
    elif 35.1 <= temperature <= 36:
        score += 1
    elif 36.1 <= temperature <= 38:
        score += 0
    elif 38.1 <= temperature <= 39:
        score += 1
    elif temperature > 39:
        score += 2

    # Score based on alertness
    if alertness.lower() == "confused or not alert":
        score += 3

    # Score based on inspired oxygen
    if inspired_oxygen.lower() == "supplemental oxygen":
        score += 2

    # Score based on performance status
    performance_status_scores = {
        "unrestricted normal activity": 0,
        "limited strenuous activity, can do light activity": 1,
        "limited activity, can self-care": 2,
        "limited self-care": 3,
        "bed/chair bound, no self-care": 4
    }
    score += performance_status_scores.get(performance_status.lower(), 0)

    # Mapping score to probability of adverse outcome using conditions
    if score in [0, 1]:
        return 0.01
    elif score in [2, 3]:
        return 0.02
    elif score == 4:
        return 0.03
    elif score == 5:
        return 0.09
    elif score == 6:
        return 0.15
    elif score == 7:
        return 0.18
    elif score == 8:
        return 0.22
    elif score == 9:
        return 0.26
    elif score == 10:
        return 0.29
    elif score == 11:
        return 0.34
    elif score == 12:
        return 0.38
    elif score == 13:
        return 0.46
    elif score == 14:
        return 0.47
    elif score == 15:
        return 0.49
    elif score == 16:
        return 0.55
    elif 17 <= score <= 25:
        return 0.59
    elif score >= 26:
        return 0.99


In [57]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest("male", 45, 20, 0.95, 80, 110, 37.0, "Alert", "Air", "Unrestricted normal activity")
Expecting:
    0.02
ok
Trying:
    priest("female", 85, 25, 0.91, 100, 85, 39.5, "Confused or not alert", "Supplemental oxygen", "Bed/chair bound, no self-care")
Expecting:
    0.59
ok
Trying:
    priest("male", 50, 12, 0.98, 60, 120, 36.5, "Alert", "Air", "Limited strenuous activity, can do light activity")
Expecting:
    0.03
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [58]:
import requests
import json
def find_hospital(age, sex, risk):
  '''This function analyzes the patient's age, sex, and risk score to identify which nearby facility to take them to in an emergency.
  It does this by using the REST web service.

  >>> find_hospital(45, 'male', 0.2)
  'Southwest Hospital and Medical Center'
  >>> find_hospital(65, 'male', 0.49)
  'Wesley Woods Geriatric Hospital'
  >>> find_hospital(29, 'female', 0.56)
  'Emory Dunwoody Medical Center'
  >>> find_hospital(19, 'female', 0.16)
  'Select Specialty Hospital - Northeast Atlanta'
  >>> find_hospital(22, 'female', 0.92)
  'Emory Dunwoody Medical Center'
  '''
  url = f'https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={risk}'
  response = requests.get(url)
  hospital_name = response.json()['hospital']
  return hospital_name

In [59]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(45, 'male', 0.2)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(65, 'male', 0.49)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok
Trying:
    find_hospital(29, 'female', 0.56)
Expecting:
    'Emory Dunwoody Medical Center'
ok
Trying:
    find_hospital(19, 'female', 0.16)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok
Trying:
    find_hospital(22, 'female', 0.92)
Expecting:
    'Emory Dunwoody Medical Center'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [60]:
import requests

def get_address(hospital_name):
    """
    Retrieves the address for the given hospital name from a JSON file.

    Args:
    hospital_name (str): Name of the hospital.

    Returns:
    str or None: The address of the hospital. Returns None if the hospital name is not found.
    >>> get_address('COOK MEDICAL CENTER')
    '706 NORTH PARRISH AVENUE'

    >>> get_address('PHOEBE PUTNEY MEMORIAL HOSPITAL')
    '417 THIRD AVENUE'

    >>> get_address("WALTON REHABILITATION HOSPITAL")
    '1355 INDEPENDENCE DRIVE'

    """
    # URL for the JSON data
    json_url = "https://hds5210-data.s3.amazonaws.com/ga_hospitals.json"

    # Send a GET request to retrieve the JSON data
    response = requests.get(json_url)

    # Parse the JSON data if the response status code is 200 (OK)
    if response.status_code == 200:
        data = response.json()

        # Search for the hospital by name
        if hospital_name in data:
            return data[hospital_name]["ADDRESS"]

    # Return None if any part of the process fails
    return None

In [61]:
import requests

def get_address(hospital_name):
    """
    Retrieves the address for the given hospital name from a JSON file.

    Args:
    hospital_name (str): Name of the hospital.

    Returns:
    str or None: The address of the hospital. Returns None if the hospital name is not found.
    >>> get_address('COOK MEDICAL CENTER')
    '706 NORTH PARRISH AVENUE'

    >>> get_address('PHOEBE PUTNEY MEMORIAL HOSPITAL')
    '417 THIRD AVENUE'

    >>> get_address("WALTON REHABILITATION HOSPITAL")
    '1355 INDEPENDENCE DRIVE'

    """
    # URL for the JSON data
    json_url = "https://hds5210-data.s3.amazonaws.com/ga_hospitals.json"

    # Send a GET request to retrieve the JSON data
    response = requests.get(json_url)

    # Parse the JSON data if the response status code is 200 (OK)
    if response.status_code == 200:
        data = response.json()

        # Search for the hospital by name
        if hospital_name in data:
            return data[hospital_name]["ADDRESS"]

    # Return None if any part of the process fails
    return None

In [62]:
import doctest
doctest.run_docstring_examples(get_address, globals(),verbose=True)

Finding tests in NoName
Trying:
    get_address('COOK MEDICAL CENTER')
Expecting:
    '706 NORTH PARRISH AVENUE'
ok
Trying:
    get_address('PHOEBE PUTNEY MEMORIAL HOSPITAL')
Expecting:
    '417 THIRD AVENUE'
ok
Trying:
    get_address("WALTON REHABILITATION HOSPITAL")
Expecting:
    '1355 INDEPENDENCE DRIVE'
ok


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [63]:
import requests as req
import json as js

"""
It compares the json data from the link with the calculated data using above functions

It returns "Data is equal" when they are equal and not equal if they are not
"""

url = "https://hds5210-data.s3.amazonaws.com/people_results.json"
headers = {'Accept' : 'application/json'}

try:
    exp= req.get(url, headers=headers)
    if exp.status_code == 200:
        first = exp.json()
        second = process_people("https://hds5210-data.s3.amazonaws.com/people.psv")
#     print("9999",second)

        firstString1 = json.dumps(first)
        secondString1 = json.dumps(second)

        firstString2 = json.loads(firstString1)
        secondString2 = json.loads(secondString1)

    if firstString2 == secondString2:
        print("Data is equal")
    else:
        print("Data is Not equal")


except Exception as err:
  print("Error in The Link")

Data is Not equal


In [64]:
import requests
import json

def risk_calculator(sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status):
    """
    Calculate the risk, hospital, and address based on the provided parameters.

    Args:
        sex (str): Gender assigned at birth, 'male' or 'female'.
        age (int): Age in years.
        breath (int): Respiratory rate in breaths per minute.
        o2sat (float): Oxygen saturation, a percent as a float between 0 and 1.
        heart (int): Heart rate in beats per minute.
        systolic (int): Systolic blood pressure in mmHg.
        temp (float): Temperature in degrees Celsius.
        alertness (str): Description of alertness, e.g., 'alert'.
        inspired (str): Description of oxygen source, e.g., 'air'.
        status (str): Description of physical activity, e.g., 'unrestricted normal activity'.

    Returns:
        tuple: A tuple containing risk score (float), hospital name (str), and hospital address (str).
    """
    # Placeholder implementation for risk calculation
    risk_score = 0.0

    # Placeholder implementation for hospital and address
    hospital_name = find_hospital(age, sex, risk_score)
    hospital_address = get_address(hospital_name)

    return risk_score, hospital_name, hospital_address

def find_hospital(age, sex, risk_score):
    """
    Find the hospital based on the patient's age, sex, and risk score.

    Args:
        age (int): The age of the patient.
        sex (str): The sex of the patient ('male' or 'female').
        risk_score (float): The calculated risk score.

    Returns:
        str: The name of the hospital.
    """
    try:
        url = f'https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={risk_score}'
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        hospital_name = data.get('hospital')
        return hospital_name
    except (requests.RequestException, json.JSONDecodeError, KeyError):
        return None

def get_address(hospital_name):
    """
    Retrieve the address of a hospital based on its name from a JSON file.

    Args:
        hospital_name (str): The name of the hospital to search for.

    Returns:
        str or None: The address of the hospital if found, or None if not found.
    """
    try:
        # Define the URL for the JSON file
        json_url = 'https://hds5210-data.s3.amazonaws.com/ga_hospitals.json'

        # Send a GET request to download the JSON data
        response = requests.get(json_url)

        # Check if the request was successful
        response.raise_for_status()

        # Load JSON data
        hospital_data = json.loads(response.text)

        # Search for the hospital by name (case-insensitive)
        for key, value in hospital_data.items():
            if hospital_name.lower() in key.lower():
                return value.get('ADDRESS')

        # If the hospital name is not found, return None
        return None
    except (requests.RequestException, json.JSONDecodeError, KeyError):
        return None

def process_people(file_location):
    """
    Process the people.psv file and calculate the risk for each individual.

    Args:
        file_location (str): The URL of the .psv file containing patient information.

    Returns:
        list: A list of dictionaries where each dictionary represents a patient's information along with calculated risk.
    """
    response = requests.get(file_location)
    data = response.text.split('\n')
    result = {}

    for line in data[1:]:
        if line.strip():  # Skips empty lines
            row = line.strip().split("|")
            patient_number = row[0]
            sex = row[1].upper()
            age = int(row[2])
            breath = int(row[3])
            o2sat = float(row[4])
            heart = int(row[5])
            systolic = int(row[6])
            temp = float(row[7])
            alertness = row[8].upper()
            inspired = row[9].upper()
            status = row[10].upper()

            # Call risk_calculator function
            risk_score, hospital, address = risk_calculator(sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status)

            if patient_number in result:
                result[patient_number].append([sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address])
            else:
                result[patient_number] = [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]

    return result

# Example usage:
file_location = 'https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh'
result = process_people(file_location)

# Save the results to a JSON file
# with open('your_results.json', 'w') as output_file:
#     json.dump(result, output_file, indent=4)

# print(result)

In [ ]:
with open('your_results.json', 'w') as output_file:
  json.dump(result, output_file, indent=4)

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [22]:
people_url = 'https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh'

In [28]:
target_result='https://hds5210-data.s3.amazonaws.com/people_results.json'

In [24]:
result.keys()

dict_keys(['E9559', 'E9385', 'E3067', 'E9422', 'E8661', 'E6235', 'E4451', 'E8433', 'E7593', 'E3296', 'E4157', 'E7702', 'E8158', 'E4795', 'E8800', 'E4855', 'E9691', 'E4535', 'E2360', 'E3447', 'E8208', 'E4428', 'E2766', 'E3888', 'E7110', 'E2668', 'E8482', 'E9619', 'E1700', 'E8585', 'E8633', 'E2117', 'E2322', 'E6912', 'E5702', 'E5513', 'E1861', 'E4809', 'E1365', 'E5984', 'E7013', 'E2331', 'E9938', 'E5121', 'E9280', 'E4762', 'E6885', 'E1695', 'E9658', 'E1961', 'E9154', 'E9417', 'E9979', 'E9316', 'E8146', 'E3093', 'E8829', 'E1453', 'E5777', 'E4564', 'E1904', 'E8746', 'E9165', 'E9305', 'E3217', 'E3127', 'E7501', 'E4997', 'E6181', 'E1073', 'E9608', 'E7663', 'E7648', 'E1883', 'E2001', 'E2418', 'E3739', 'E8351', 'E2882', 'E1902', 'E2155', 'E4345', 'E4194', 'E5211', 'E5488', 'E8560', 'E2995', 'E3422', 'E7707', 'E3119', 'E1419', 'E7748', 'E7251', 'E1235', 'E5361', 'E7130', 'E5665', 'E9957', 'E10000'])

In [29]:
r=requests.request('GET',url=target_result)
r.status_code

200

In [32]:
type(json.loads(r.text))

dict

In [41]:
import json

# def process_people(file_location):
#     try:
#         response = requests.request('GET',file_location)
#         if response.status_code == 200:
#             return json.loads(response.text)
#         else:
#             return None
#     except (requests.RequestException, json.JSONDecodeError, KeyError):
#         return None


people_results = result.copy()

response_expected = json.loads(requests.request('GET',url=target_result).text)



In [37]:
r1=requests.request('GET',people_url)
r1.text

'patient|sex|age|breath|o2 sat|heart|systolic bp|temp|alertness|inspired|status\nE9559|FEMALE|40|24|0.96|105|115|34.9|ALERT|AIR|unrestricted normal activity\nE9385|Female|51|19|0.99|90|91|38.1|ALERT|supplemental oxygen|unrestricted normal activity\nE3067|female|40|29|0.96|105|95|38.1|ALERT|air|unrestricted normal activity\nE9422|FEMALE|66|19|0.96|135|115|38.1|ALERT|AIR|bed/chair bound, no self-care\nE8661|MALE|18|22|0.95|105|115|36.3|Confused or Not Alert|air|unrestricted normal activity\nE6235|Male|16|23|0.96|135|115|39.2|ALERT|supplemental oxygen|unrestricted normal activity\nE4451|MALE|40|8|0.96|135|115|38.1|ALERT|AIR|unrestricted normal activity\nE8433|FEMALE|40|23|0.99|89|91|38.1|ALERT|AIR|limited self-care\nE7593|FEMALE|80|23|0.96|105|115|38.1|ALERT|supplemental oxygen|limited activity, can self-care\nE3296|MALE|51|23|0.95|105|115|38.1|ALERT|AIR|unrestricted normal activity\nE4157|FEMALE|40|23|0.96|41|115|35.4|ALERT|AIR|unrestricted normal activity\nE7702|FEMALE|40|22|0.96|105|11

In [49]:
people_ids=people_results.keys()
expected_ids=response_expected.keys()

# checking if key count and keys match - intersection and counts
if len(people_ids)==len(expected_ids) :
    print("ID count match, proceeding further checks")
    if len(set(people_ids).difference(set(expected_ids)))==0:
        print("All Keys or IDs match")
    else:
        print("Count matches but there is mismatch in IDs")
else:
    print("id count doesnt match, so results dont match no furthersteps needed")



ID count match, proceeding further checks
All Keys or IDs match, checking if values match... Stay tuned
No Match


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---